# load file 

In [1]:
import json
def load_data(file_path):
        '''load json file and return data'''
        with open(file_path, 'r', encoding='utf-8') as file:
            data = json.load(file)
            return data

# preprocessing pipeline

In [2]:
from camel_pipeline import preprocess_text
text="تُمكّنك هذه الخدمة من تفعيل بطاقتك التموينية. الأسرة"
pre=preprocess_text(text)
pre = " ".join(pre)



# extract keywords from string
- list of doument 
-

In [3]:
from camel_pipeline import extract_keywords_tfidf
extract_keywords_tfidf([pre])

[['تمكنك', 'تفعيل']]

# extract words 
- بنجمع كله مع بعض وناخد الحذر لو فيه اكتر من terms علشان كدا بنعملها join 
-بعدين بنضيف full text ل كل خدمه 
-وبعدين بنضيف short text عباره عن نفس اللي فات بس من غير terms 
-وبعدين بنتضرب علي full text علشان نكتر documents كتير علشان tfidf يبقي accurate اكتر 
-وبعدين يعمل transform علي shortwords ويطلع اكتر كلمتين مهمين 

In [1]:
import json
from camel_pipeline import enrich_services_with_texts, extract_keywords_from_short_texts

# Step 1: Load services from file
with open("scraped_data.json", "r", encoding="utf-8") as f:
    text = json.load(f)

# Step 2: Enrich services with full_text and short_text
enriched_services = enrich_services_with_texts(text)

# Step 3: Extract top 2 keywords from short_text using full_text TF-IDF
services_with_keywords = extract_keywords_from_short_texts(enriched_services)

# Step 4: Save result
with open("services_with_keywords.json", "w", encoding="utf-8") as f:
    json.dump(services_with_keywords, f, ensure_ascii=False, indent=4)